<a href="https://colab.research.google.com/github/MayaBO007/Analysis-scripts/blob/main/update_subs_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
import json

# Authenticate and create client
auth.authenticate_user()
credentials = GoogleCredentials.get_application_default()
client = gspread.authorize(credentials)

# Open the Google Sheet
sheet = client.open("subjects_info").sheet1

# Create a list to store batch update requests
batch_update = []

# Get the values from the "WorkerId" column
worker_ids = sheet.col_values(1)

# Iterate over the WorkerIds and update the corresponding columns
# Start from row 2 (skip header row)
for row_index, worker_id in enumerate(worker_ids[1:], start=2):
    # Check if WorkerId exists
    if worker_id:
        # Open the JSON file
        filename = f"results({10}).json"  # Update the filename accordingly
        with open(filename, "r") as json_file:
            data_list = json.load(json_file)

            for data in data_list:
                # Check if "worker_id" and "doneTest2" match the condition
                if data.get("worker_id") == worker_id and data.get("doneTest2") == "doneTest2":
                    # Update "isExperimentDone?" column to 1
                    batch_update.append({
                        'range': f'E{row_index}',
                        'values': [['1']]
                    })
                    # Check if "totalBlues" and "totalReds" exist and are not empty
                    if "totalBlues" in data and data["totalBlues"]:
                        total_blues = int(data["totalBlues"])
                    else:
                        total_blues = 0

                    if "totalReds" in data and data["totalReds"]:
                        total_reds = int(data["totalReds"])
                    else:
                        total_reds = 0

                    # Calculate the sum of "totalBlues" and "totalReds"
                    total_wins = total_blues + total_reds

                    # Update "totalWins" column
                    batch_update.append({
                        'range': f'F{row_index}',
                        'values': [[str(total_wins)]]
                    })

# Perform the batch update
if batch_update:
    request_body = {
        'requests': [{'updateCells': {'rows': batch_update}}]
    }
    sheet.batch_update(request_body)
